<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Todo: é preciso fazer a varredura de todos os diretórios src contendo .java

# Checa ambiente

In [ ]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
!python3 --version

Python 3.10.12


In [ ]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (107 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120831 files and directories currently install

# Funcões

In [ ]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd kafka && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [ ]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

#core/src/main/java
def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['KAFKA'] = tag + 'core/src/main/java'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $KAFKA -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [ ]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

In [ ]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [ ]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [ ]:
!git clone https://github.com/apache/kafka.git

Cloning into 'kafka'...
remote: Enumerating objects: 345347, done.
remote: Total 345347 (delta 0), reused 0 (delta 0), pack-reused 345347
Receiving objects: 100% (345347/345347), 161.03 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (166525/166525), done.


In [ ]:
!cd kafka && find . -name "src"

./core/src
./storage/src
./storage/api/src
./clients/src
./trogdor/src
./raft/src
./connect/test-plugins/src
./connect/basic-auth-extension/src
./connect/runtime/src
./connect/file/src
./connect/mirror/src
./connect/mirror-client/src
./connect/json/src
./connect/transforms/src
./connect/api/src
./server-common/src
./metadata/src
./group-coordinator/src
./log4j-appender/src
./tools/src
./tools/tools-api/src
./examples/src
./jmh-benchmarks/src
./streams/upgrade-system-tests-0110/src
./streams/src
./streams/upgrade-system-tests-32/src
./streams/upgrade-system-tests-30/src
./streams/upgrade-system-tests-11/src
./streams/upgrade-system-tests-27/src
./streams/upgrade-system-tests-23/src
./streams/upgrade-system-tests-20/src
./streams/quickstart/java/src
./streams/quickstart/java/src/main/resources/archetype-resources/src
./streams/streams-scala/src
./streams/upgrade-system-tests-0101/src
./streams/upgrade-system-tests-26/src
./streams/test-utils/src
./streams/upgrade-system-tests-33/src
./st

In [ ]:
!cd /content/kafka && find . -name "*.java" -exec dirname {} \; | sort -u

./clients/src/main/java/org/apache/kafka/clients
./clients/src/main/java/org/apache/kafka/clients/admin
./clients/src/main/java/org/apache/kafka/clients/admin/internals
./clients/src/main/java/org/apache/kafka/clients/consumer
./clients/src/main/java/org/apache/kafka/clients/consumer/internals
./clients/src/main/java/org/apache/kafka/clients/consumer/internals/events
./clients/src/main/java/org/apache/kafka/clients/producer
./clients/src/main/java/org/apache/kafka/clients/producer/internals
./clients/src/main/java/org/apache/kafka/common
./clients/src/main/java/org/apache/kafka/common/acl
./clients/src/main/java/org/apache/kafka/common/annotation
./clients/src/main/java/org/apache/kafka/common/cache
./clients/src/main/java/org/apache/kafka/common/compress
./clients/src/main/java/org/apache/kafka/common/config
./clients/src/main/java/org/apache/kafka/common/config/internals
./clients/src/main/java/org/apache/kafka/common/config/provider
./clients/src/main/java/org/apache/kafka/common/co

### Pega a lista de tags

In [ ]:
lista_tags_kafka = !cd kafka && git tag

In [ ]:
pattern_to_extract = r'3.'
get_elements_with_pattern(lista_tags_kafka, pattern_to_extract)

['0.11.0.3-rc0',
 '2.3.0',
 '2.3.0-rc1',
 '2.3.0-rc2',
 '2.3.0-rc3',
 '2.3.1',
 '2.3.1-rc0',
 '2.3.1-rc1',
 '2.3.1-rc2',
 '2.6.3-rc0',
 '3.0.0',
 '3.0.0-rc0',
 '3.0.0-rc1',
 '3.0.0-rc2',
 '3.0.1',
 '3.0.1-rc0',
 '3.0.2',
 '3.0.2-rc0',
 '3.1.0',
 '3.1.0-rc0',
 '3.1.0-rc1',
 '3.1.1',
 '3.1.1-rc0',
 '3.1.1-rc1',
 '3.1.2',
 '3.1.2-rc0',
 '3.2.0',
 '3.2.0-rc0',
 '3.2.0-rc1',
 '3.2.1',
 '3.2.1-rc2',
 '3.2.1-rc3',
 '3.2.2',
 '3.2.2-rc0',
 '3.2.3',
 '3.2.3-rc0',
 '3.3.0',
 '3.3.0-rc1',
 '3.3.0-rc2',
 '3.3.1',
 '3.3.1-rc0',
 '3.3.2',
 '3.3.2-rc0',
 '3.3.2-rc1',
 '3.4.0',
 '3.4.0-rc0',
 '3.4.0-rc1',
 '3.4.0-rc2',
 '3.4.1',
 '3.4.1-rc0',
 '3.4.1-rc1',
 '3.4.1-rc2',
 '3.4.1-rc3',
 '3.5.0',
 '3.5.0-rc0',
 '3.5.0-rc1',
 '3.5.1',
 '3.5.1-rc0',
 '3.5.1-rc1']

# Análise da tag kafka_3_0_1

In [ ]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_3_0_1 = get_commit_tag(tag='3.0.1')
cria_pasta_n_commit(repositorio='kafka', pasta_n_commit='commit_tag_3_0_1')
checkout_n_commit('commit_tag_3_0_1', commit_tag_3_0_1[0])

Checkout do repositorio no commit 8e30984f43e64d8b9c2715c431a8906c98d26b86 realizado com sucesso!


In [ ]:
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/core/src/main/java/kafka -o commit_tag_3_0_1/core/src/main/java/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/core/src/main/scala/kafka -o commit_tag_3_0_1/core/src/main/scala/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/core/src/test/java/kafka -o commit_tag_3_0_1/core/src/test/java/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/clients/src/main/java/org/apache/kafka -o commit_tag_3_0_1/clients/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/clients/src/test/java/org/apache/kafka -o commit_tag_3_0_1/clients/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/api/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/api/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/basic-auth-extension/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/basic-auth-extension/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/file/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/file/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/json/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/json/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/mirror-client/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/mirror-client/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/mirror/src/main/java/org/apache/kafka -o commit_tag_3_0_1/connect/mirror/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/runtime/src/main/java/org/apache/kafka -o commit_tag_3_0_1/connect/runtime/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/runtime/src/test/java/org/apache/kafka -o commit_tag_3_0_1/connect/runtime/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins -o commit_tag_3_0_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/test-plugins/src/main/java/org/apache/kafka -o commit_tag_3_0_1/connect/test-plugins/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/connect/transforms/src/main/java/org/apache/kafka -o commit_tag_3_0_1/connect/transforms/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/generator/src/main/java/org/apache/kafka -o commit_tag_3_0_1/generator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/generator/src/test/java/org/apache/kafka -o commit_tag_3_0_1/generator/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/group-coordinator/src/main/java/org/apache/kafka -o commit_tag_3_0_1/group-coordinator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/group-coordinator/src/main/java/org/apache/kafka -o commit_tag_3_0_1/group-coordinator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/jmh-benchmarks/src/main/java/org/apache/kafka -o commit_tag_3_0_1/jmh-benchmarks/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/log4j-appender/src/main/java/org/apache/kafka -o commit_tag_3_0_1/log4j-appender/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/metadata/src/main/java/org/apache/kafka -o commit_tag_3_0_1/metadata/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/metadata/src/test/java/org/apache/kafka -o commit_tag_3_0_1/metadata/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/raft/src/main/java/org/apache/kafka -o commit_tag_3_0_1/raft/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/server-common/src/main/java/org/apache/kafka -o commit_tag_3_0_1/server-common/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/shell/src/main/java/org/apache/kafka -o commit_tag_3_0_1/shell/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/storage/api/src/main/java/org/apache/kafka -o commit_tag_3_0_1/storage/api/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/storage/src/main/java/org/apache/kafka -o commit_tag_3_0_1/storage/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/streams/examples/src/main/java/org/apache/kafka -o commit_tag_3_0_1/streams/examples/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java -o commit_tag_3_0_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/streams/src/main/java/org/apache/kafka -o commit_tag_3_0_1/streams/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/streams/test-utils/src/main/java/org/apache/kafka -o commit_tag_3_0_1/streams/test-utils/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/tools/src/main/java/org/apache/kafka -o commit_tag_3_0_1/tools/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/tools/tools-api/src/main/java/org/apache/kafka -o commit_tag_3_0_1/tools/tools-api/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/trogdor/src/main/java/org/apache/kafka -o commit_tag_3_0_1/trogdor/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_0_1/trogdor/src/test/java/org/apache/kafka -o commit_tag_3_0_1/trogdor/src/test/java/org/apache/kafka

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "ffffffffffffff80"
Exception while parsing literal number (during Magic Number detection). For input string: "ffffffffffffff80"
Exception while parsing literal number (during Magic Number detection). For input string: "ffffffffffffff80"
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Nu

In [ ]:
!ls -liat commit_tag_3_0_1/core/src/main/java/kafka/designCodeSmells.csv

2633047 -rw-r--r-- 1 root root 110 Aug 27 20:49 commit_tag_3_0_1/core/src/main/java/kafka/designCodeSmells.csv


In [ ]:
df_design_smells1 = pd.read_csv('commit_tag_3_0_1/core/src/main/java/kafka/designCodeSmells.csv')
df_design_smells2 = pd.read_csv('commit_tag_3_0_1/core/src/main/scala/kafka/designCodeSmells.csv')
df_design_smells3 = pd.read_csv('commit_tag_3_0_1/core/src/test/java/kafka/designCodeSmells.csv')
df_design_smells4 = pd.read_csv('commit_tag_3_0_1/clients/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells5 = pd.read_csv('commit_tag_3_0_1/clients/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells6= pd.read_csv('commit_tag_3_0_1/connect/api/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells7 = pd.read_csv('commit_tag_3_0_1/connect/basic-auth-extension/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells8 = pd.read_csv('commit_tag_3_0_1/connect/file/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells9 = pd.read_csv('commit_tag_3_0_1/connect/json/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells10 = pd.read_csv('commit_tag_3_0_1/connect/mirror-client/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells11 = pd.read_csv('commit_tag_3_0_1/connect/mirror/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells12 = pd.read_csv('commit_tag_3_0_1/connect/runtime/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells13 = pd.read_csv('commit_tag_3_0_1/connect/runtime/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells14 = pd.read_csv('commit_tag_3_0_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins/designCodeSmells.csv')

df_design_smells16 = pd.read_csv('commit_tag_3_0_1/connect/transforms/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells17 = pd.read_csv('commit_tag_3_0_1/generator/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells18 = pd.read_csv('commit_tag_3_0_1/generator/src/test/java/org/apache/kafka/designCodeSmells.csv')


df_design_smells21 = pd.read_csv('commit_tag_3_0_1/jmh-benchmarks/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells22 = pd.read_csv('commit_tag_3_0_1/log4j-appender/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells23 = pd.read_csv('commit_tag_3_0_1/metadata/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells24 = pd.read_csv('commit_tag_3_0_1/metadata/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells25 = pd.read_csv('commit_tag_3_0_1/raft/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells26 = pd.read_csv('commit_tag_3_0_1/server-common/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells27 = pd.read_csv('commit_tag_3_0_1/shell/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells28 = pd.read_csv('commit_tag_3_0_1/storage/api/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells29 = pd.read_csv('commit_tag_3_0_1/storage/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells30 = pd.read_csv('commit_tag_3_0_1/streams/examples/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells31 = pd.read_csv('commit_tag_3_0_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java/designCodeSmells.csv')
df_design_smells32 = pd.read_csv('commit_tag_3_0_1/streams/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells33 = pd.read_csv('commit_tag_3_0_1/streams/test-utils/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells34 = pd.read_csv('commit_tag_3_0_1/tools/src/main/java/org/apache/kafka/designCodeSmells.csv')

df_design_smells36 = pd.read_csv('commit_tag_3_0_1/trogdor/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells37 = pd.read_csv('commit_tag_3_0_1/trogdor/src/test/java/org/apache/kafka/designCodeSmells.csv')

In [ ]:
dataframes_i = [df_design_smells1, df_design_smells2, df_design_smells3, df_design_smells4, df_design_smells5, df_design_smells6, df_design_smells7, df_design_smells8, df_design_smells9, df_design_smells10, df_design_smells11, df_design_smells12, df_design_smells13, df_design_smells14, df_design_smells16, df_design_smells17, df_design_smells18,  df_design_smells21, df_design_smells22, df_design_smells23, df_design_smells24, df_design_smells25, df_design_smells26, df_design_smells27, df_design_smells28, df_design_smells29, df_design_smells30, df_design_smells31, df_design_smells32, df_design_smells33, df_design_smells34, df_design_smells36, df_design_smells37]

In [ ]:
# Merge the dataframes
merged_dataframe_i = pd.concat(dataframes_i, axis=0, ignore_index=True)

df_design_smells_final_i = merged_dataframe_i.copy()
df_design_smells_final_i

,Project Name,Package Name,Type Name,Code Smell
0,kafka,kafka.metrics,KafkaYammerMetrics,Deficient Encapsulation
1,kafka,kafka.testkit,Builder,Unutilized Abstraction
2,kafka,kafka.testkit,Builder,Unutilized Abstraction
3,kafka,kafka.testkit,Builder,Unutilized Abstraction
4,kafka,kafka.testkit,TestKitNodes,Unexploited Encapsulation
...,...,...,...,...
3302,kafka,org.apache.kafka.trogdor.task,SampleTaskController,Unutilized Abstraction
3303,kafka,org.apache.kafka.trogdor.task,TaskSpecTest,Unutilized Abstraction
3304,kafka,org.apache.kafka.trogdor.task,SampleTaskWorker,Unutilized Abstraction
3305,kafka,org.apache.kafka.trogdor.rest,RestExceptionMapperTest,Unutilized Abstraction


In [ ]:
df_kafka_criticos = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/piloto/main/criticos/kafka.csv')
filenames = df_kafka_criticos.File.to_list()
filenames

['StreamThread.java',
 'KafkaConsumer.java',
 'KStream.java',
 'KafkaAdminClient.java',
 'StreamTask.java',
 'Fetcher.java',
 'KafkaStreams.java',
 'TaskManager.java',
 'KStreamImpl.java',
 'KafkaProducer.java',
 'StreamsConfig.java',
 'ConsumerCoordinator.java',
 'Sender.java',
 'AbstractCoordinator.java',
 'KTableImpl.java',
 'KTable.java',
 'StreamsPartitionAssignor.java',
 'NetworkClient.java',
 'InternalTopologyBuilder.java',
 'Protocol.java']

In [ ]:
df_design_smells_final_i['Type Name'].unique()

array(['KafkaYammerMetrics', 'Builder', 'TestKitNodes', ...,
       'SampleTaskWorker', 'RestExceptionMapperTest', 'MockKibosh'],
      dtype=object)

In [ ]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
df_temp_kafka = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/kafka-treemap/main/results/kafka_java_20_critical.csv')
df_temp_kafka

,Unnamed: 0,File,Frequency,lines_modified,fcxflc
0,452,StreamThread.java,323,14137,4566251
1,168,KafkaConsumer.java,287,10142,2910754
2,403,KStream.java,121,18622,2253262
3,1372,KafkaAdminClient.java,191,10647,2033577
4,451,StreamTask.java,212,7250,1537000
5,233,Fetcher.java,195,7834,1527630
6,760,KafkaStreams.java,201,7232,1453632
7,1555,TaskManager.java,158,7917,1250886
8,419,KStreamImpl.java,134,8558,1146772
9,35,KafkaProducer.java,218,4765,1038770


In [ ]:
lista_classes_criticas = df_temp_kafka.File.tolist()
lista_classes_criticas

['StreamThread.java',
 'KafkaConsumer.java',
 'KStream.java',
 'KafkaAdminClient.java',
 'StreamTask.java',
 'Fetcher.java',
 'KafkaStreams.java',
 'TaskManager.java',
 'KStreamImpl.java',
 'KafkaProducer.java',
 'StreamsConfig.java',
 'ConsumerCoordinator.java',
 'Sender.java',
 'AbstractCoordinator.java',
 'KTableImpl.java',
 'KTable.java',
 'StreamsPartitionAssignor.java',
 'NetworkClient.java',
 'InternalTopologyBuilder.java',
 'Protocol.java']

In [ ]:
lista_nomes = []
for each in lista_classes_criticas:
  each = each.replace('.java', '')
  lista_nomes.append(each)

In [ ]:
pesquisa = df_design_smells_final_i['Type Name'].str.contains('|'.join(lista_nomes))

In [ ]:
df_critical_classes = df_design_smells_final_i[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
28,kafka,org.apache.kafka.clients,NetworkClient,Deficient Encapsulation
29,kafka,org.apache.kafka.clients,NetworkClient,Cyclic-Dependent Modularization
30,kafka,org.apache.kafka.clients,NetworkClient,Insufficient Modularization
85,kafka,org.apache.kafka.clients.admin,KafkaAdminClient,Unutilized Abstraction
86,kafka,org.apache.kafka.clients.admin,KafkaAdminClient,Unexploited Encapsulation
...,...,...,...,...
3045,kafka,org.apache.kafka.streams.processor.internals.a...,StreamsAssignmentProtocolVersions,Unutilized Abstraction
3046,kafka,org.apache.kafka.streams.processor.internals.a...,StreamsAssignmentProtocolVersions,Deficient Encapsulation
3199,kafka,org.apache.kafka.trogdor.coordinator,TaskManager,Deficient Encapsulation
3200,kafka,org.apache.kafka.trogdor.coordinator,TaskManager,Cyclic-Dependent Modularization


In [ ]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
29,kafka,org.apache.kafka.clients,NetworkClient,Cyclic-Dependent Modularization
89,kafka,org.apache.kafka.clients.admin,KafkaAdminClient,Cyclic-Dependent Modularization
275,kafka,org.apache.kafka.clients.consumer.internals,AbstractCoordinator,Cyclic-Dependent Modularization
309,kafka,org.apache.kafka.clients.consumer.internals,ConsumerNetworkClient,Cyclic-Dependent Modularization
2465,kafka,org.apache.kafka.streams,StreamsConfig,Cyclic-Dependent Modularization
2728,kafka,org.apache.kafka.streams.kstream.internals,CogroupedKStreamImpl,Cyclic-Dependent Modularization
2743,kafka,org.apache.kafka.streams.kstream.internals,KStreamImpl,Cyclic-Dependent Modularization
2790,kafka,org.apache.kafka.streams.kstream.internals,KStreamImplJoin,Cyclic-Dependent Modularization
2799,kafka,org.apache.kafka.streams.kstream.internals,KTableKTableAbstractJoin,Cyclic-Dependent Modularization
2816,kafka,org.apache.kafka.streams.kstream.internals,KTableImpl,Cyclic-Dependent Modularization


In [ ]:
final_critical_classes = df_critical_classes.TypeName.unique().tolist()
final_critical_classes

['NetworkClient',
 'KafkaAdminClient',
 'AbstractCoordinator',
 'ConsumerNetworkClient',
 'StreamsConfig',
 'CogroupedKStreamImpl',
 'KStreamImpl',
 'KStreamImplJoin',
 'KTableKTableAbstractJoin',
 'KTableImpl',
 'StreamThread',
 'InternalTopologyBuilder',
 'StreamTask',
 'StreamsPartitionAssignor',
 'TaskManager']

In [ ]:
df_analise_kafka_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_kafka_critical_classes.to_csv('commit_tag_3_0_1/analise_kafka_classes_criticas.csv', index=False)
df_design_smells_final_i.to_csv('commit_tag_3_0_1/analise_designite.csv', index=False)

# Análise da tag kafka_3_5_1

In [ ]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_3_5_1 = get_commit_tag(tag='3.5.1')
cria_pasta_n_commit(repositorio='kafka', pasta_n_commit='commit_tag_3_5_1')
checkout_n_commit('commit_tag_3_5_1', commit_tag_3_5_1[0])

Checkout do repositorio no commit 2c6fb6c54472e90ae17439e62540ef3cb0426fe3 realizado com sucesso!


In [ ]:
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/core/src/main/java -o commit_tag_3_5_1

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [ ]:
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/core/src/main/java/kafka -o commit_tag_3_5_1/core/src/main/java/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/core/src/main/scala/kafka -o commit_tag_3_5_1/core/src/main/scala/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/core/src/test/java/kafka -o commit_tag_3_5_1/core/src/test/java/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/clients/src/main/java/org/apache/kafka -o commit_tag_3_5_1/clients/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/clients/src/test/java/org/apache/kafka -o commit_tag_3_5_1/clients/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/api/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/api/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/basic-auth-extension/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/basic-auth-extension/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/file/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/file/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/json/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/json/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/mirror-client/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/mirror-client/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/mirror/src/main/java/org/apache/kafka -o commit_tag_3_5_1/connect/mirror/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/runtime/src/main/java/org/apache/kafka -o commit_tag_3_5_1/connect/runtime/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/runtime/src/test/java/org/apache/kafka -o commit_tag_3_5_1/connect/runtime/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins -o commit_tag_3_5_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/test-plugins/src/main/java/org/apache/kafka -o commit_tag_3_5_1/connect/test-plugins/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/connect/transforms/src/main/java/org/apache/kafka -o commit_tag_3_5_1/connect/transforms/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/generator/src/main/java/org/apache/kafka -o commit_tag_3_5_1/generator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/generator/src/test/java/org/apache/kafka -o commit_tag_3_5_1/generator/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/group-coordinator/src/main/java/org/apache/kafka -o commit_tag_3_5_1/group-coordinator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/group-coordinator/src/main/java/org/apache/kafka -o commit_tag_3_5_1/group-coordinator/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/jmh-benchmarks/src/main/java/org/apache/kafka -o commit_tag_3_5_1/jmh-benchmarks/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/log4j-appender/src/main/java/org/apache/kafka -o commit_tag_3_5_1/log4j-appender/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/metadata/src/main/java/org/apache/kafka -o commit_tag_3_5_1/metadata/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/metadata/src/test/java/org/apache/kafka -o commit_tag_3_5_1/metadata/src/test/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/raft/src/main/java/org/apache/kafka -o commit_tag_3_5_1/raft/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/server-common/src/main/java/org/apache/kafka -o commit_tag_3_5_1/server-common/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/shell/src/main/java/org/apache/kafka -o commit_tag_3_5_1/shell/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/storage/api/src/main/java/org/apache/kafka -o commit_tag_3_5_1/storage/api/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/storage/src/main/java/org/apache/kafka -o commit_tag_3_5_1/storage/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/streams/examples/src/main/java/org/apache/kafka -o commit_tag_3_5_1/streams/examples/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java -o commit_tag_3_5_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/streams/src/main/java/org/apache/kafka -o commit_tag_3_5_1/streams/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/streams/test-utils/src/main/java/org/apache/kafka -o commit_tag_3_5_1/streams/test-utils/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/tools/src/main/java/org/apache/kafka -o commit_tag_3_5_1/tools/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/tools/tools-api/src/main/java/org/apache/kafka -o commit_tag_3_5_1/tools/tools-api/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/trogdor/src/main/java/org/apache/kafka -o commit_tag_3_5_1/trogdor/src/main/java/org/apache/kafka
!java -jar DesigniteJava.jar -i commit_tag_3_5_1/trogdor/src/test/java/org/apache/kafka -o commit_tag_3_5_1/trogdor/src/test/java/org/apache/kafka

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string

In [ ]:
df_design_smells_f_1 = pd.read_csv('commit_tag_3_5_1/core/src/main/java/kafka/designCodeSmells.csv')
df_design_smells_f_2 = pd.read_csv('commit_tag_3_5_1/core/src/main/scala/kafka/designCodeSmells.csv')
df_design_smells_f_3 = pd.read_csv('commit_tag_3_5_1/core/src/test/java/kafka/designCodeSmells.csv')
df_design_smells_f_4 = pd.read_csv('commit_tag_3_5_1/clients/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_5 = pd.read_csv('commit_tag_3_5_1/clients/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_6= pd.read_csv('commit_tag_3_5_1/connect/api/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_7 = pd.read_csv('commit_tag_3_5_1/connect/basic-auth-extension/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_8 = pd.read_csv('commit_tag_3_5_1/connect/file/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_9 = pd.read_csv('commit_tag_3_5_1/connect/json/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_10 = pd.read_csv('commit_tag_3_5_1/connect/mirror-client/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_11 = pd.read_csv('commit_tag_3_5_1/connect/mirror/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_12 = pd.read_csv('commit_tag_3_5_1/connect/runtime/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_13 = pd.read_csv('commit_tag_3_5_1/connect/runtime/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_14 = pd.read_csv('commit_tag_3_5_1/connect/runtime/src/test/resources/test-plugins/aliased-static-field/test/plugins/designCodeSmells.csv')

df_design_smells_f_16 = pd.read_csv('commit_tag_3_5_1/connect/transforms/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_17 = pd.read_csv('commit_tag_3_5_1/generator/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_18 = pd.read_csv('commit_tag_3_5_1/generator/src/test/java/org/apache/kafka/designCodeSmells.csv')


df_design_smells_f_21 = pd.read_csv('commit_tag_3_5_1/jmh-benchmarks/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_22 = pd.read_csv('commit_tag_3_5_1/log4j-appender/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_23 = pd.read_csv('commit_tag_3_5_1/metadata/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_24 = pd.read_csv('commit_tag_3_5_1/metadata/src/test/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_25 = pd.read_csv('commit_tag_3_5_1/raft/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_26 = pd.read_csv('commit_tag_3_5_1/server-common/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_27 = pd.read_csv('commit_tag_3_5_1/shell/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_28 = pd.read_csv('commit_tag_3_5_1/storage/api/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_29 = pd.read_csv('commit_tag_3_5_1/storage/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_30 = pd.read_csv('commit_tag_3_5_1/streams/examples/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_31 = pd.read_csv('commit_tag_3_5_1/streams/quickstart/java/src/main/resources/archetype-resources/src/main/java/designCodeSmells.csv')
df_design_smells_f_32 = pd.read_csv('commit_tag_3_5_1/streams/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_33 = pd.read_csv('commit_tag_3_5_1/streams/test-utils/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_34 = pd.read_csv('commit_tag_3_5_1/tools/src/main/java/org/apache/kafka/designCodeSmells.csv')

df_design_smells_f_36 = pd.read_csv('commit_tag_3_5_1/trogdor/src/main/java/org/apache/kafka/designCodeSmells.csv')
df_design_smells_f_37 = pd.read_csv('commit_tag_3_5_1/trogdor/src/test/java/org/apache/kafka/designCodeSmells.csv')

In [ ]:
dataframes_f = [df_design_smells_f_1, df_design_smells_f_2, df_design_smells_f_3, df_design_smells_f_4, df_design_smells_f_5, df_design_smells_f_6, df_design_smells_f_7, df_design_smells_f_8, df_design_smells_f_9, df_design_smells_f_10, df_design_smells_f_11, df_design_smells_f_12, df_design_smells_f_13, df_design_smells_f_14, df_design_smells_f_16, df_design_smells_f_17, df_design_smells_f_18,  df_design_smells_f_21, df_design_smells_f_22, df_design_smells_f_23, df_design_smells_f_24, df_design_smells_f_25, df_design_smells_f_26, df_design_smells_f_27, df_design_smells_f_28, df_design_smells_f_29, df_design_smells_f_30, df_design_smells_f_31, df_design_smells_f_32, df_design_smells_f_33, df_design_smells_f_34, df_design_smells_f_36, df_design_smells_f_37]

In [ ]:
# Merge the dataframes
merged_dataframe_f = pd.concat(dataframes_f, axis=0, ignore_index=True)

df_design_smells_final_f = merged_dataframe_f.copy()
df_design_smells_final_f

,Project Name,Package Name,Type Name,Code Smell
0,kafka,kafka.server.builders,KafkaApisBuilder,Unutilized Abstraction
1,kafka,kafka.server.builders,KafkaApisBuilder,Insufficient Modularization
2,kafka,kafka.server.builders,ReplicaManagerBuilder,Unutilized Abstraction
3,kafka,kafka.server.builders,ReplicaManagerBuilder,Insufficient Modularization
4,kafka,kafka.server.builders,LogManagerBuilder,Unutilized Abstraction
...,...,...,...,...
3943,kafka,org.apache.kafka.trogdor.task,SampleTaskController,Unutilized Abstraction
3944,kafka,org.apache.kafka.trogdor.task,TaskSpecTest,Unutilized Abstraction
3945,kafka,org.apache.kafka.trogdor.task,SampleTaskWorker,Unutilized Abstraction
3946,kafka,org.apache.kafka.trogdor.rest,RestExceptionMapperTest,Unutilized Abstraction


In [ ]:
nomes_classes_criticas = []
for each in lista_classes_criticas:
  each = each.replace('.java', '')
  nomes_classes_criticas.append(each)

nomes_classes_criticas

['StreamThread',
 'KafkaConsumer',
 'KStream',
 'KafkaAdminClient',
 'StreamTask',
 'Fetcher',
 'KafkaStreams',
 'TaskManager',
 'KStreamImpl',
 'KafkaProducer',
 'StreamsConfig',
 'ConsumerCoordinator',
 'Sender',
 'AbstractCoordinator',
 'KTableImpl',
 'KTable',
 'StreamsPartitionAssignor',
 'NetworkClient',
 'InternalTopologyBuilder',
 'Protocol']

In [ ]:
pesquisa_final_f = df_design_smells_final_f['Type Name'].str.contains('|'.join(nomes_classes_criticas))

In [ ]:
df_critical_classes_final_f = df_design_smells_final_f[pesquisa_final_f]
df_critical_classes_final_f.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final_f = df_critical_classes_final_f.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [ ]:
df_critical_classes_final_f

,ProjectName,PackageName,TypeName,CodeSmell
45,kafka,org.apache.kafka.clients,NetworkClient,Cyclic-Dependent Modularization
110,kafka,org.apache.kafka.clients.admin,KafkaAdminClient,Cyclic-Dependent Modularization
320,kafka,org.apache.kafka.clients.consumer.internals,AbstractCoordinator,Cyclic-Dependent Modularization
363,kafka,org.apache.kafka.clients.consumer.internals,ConsumerNetworkClient,Cyclic-Dependent Modularization
2999,kafka,org.apache.kafka.streams,StreamsConfig,Cyclic-Dependent Modularization
3310,kafka,org.apache.kafka.streams.kstream.internals,CogroupedKStreamImpl,Cyclic-Dependent Modularization
3327,kafka,org.apache.kafka.streams.kstream.internals,KStreamImpl,Cyclic-Dependent Modularization
3374,kafka,org.apache.kafka.streams.kstream.internals,KStreamImplJoin,Cyclic-Dependent Modularization
3384,kafka,org.apache.kafka.streams.kstream.internals,KTableKTableAbstractJoin,Cyclic-Dependent Modularization
3401,kafka,org.apache.kafka.streams.kstream.internals,KTableImpl,Cyclic-Dependent Modularization


In [ ]:
df_analise_designite_critical_classes_final_f = df_critical_classes_final_f.sort_values(by='TypeName')
df_analise_designite_critical_classes_final_f.to_csv('commit_tag_3_5_1/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final_f.to_csv('commit_tag_3_5_1/analise_designite.csv', index=False)